In [1]:
from compressor.average_streamline.compressor import Compressor
from compressor.average_streamline.stage_tools import Stage
from compressor.average_streamline.dist_tools import QuadraticBezier, QuadraticCurve, QuadraticSpline
import ipywidgets as widgets
from turbine.average_streamline.turbine import Turbine
from gas_turbine_cycle.gases import Air
import numpy as np
from scipy.interpolate import interp1d
from IPython.display import display
import matplotlib.pyplot as plt
import pickle
import os
import config

%matplotlib inline

In [2]:
with open(os.path.join(config.output_dirname, config.cycle_results), 'rb') as f:
    units = pickle.load(f)[0]

turb_load = units['turb_load']
cycle_compressor = units['compressor']
N_e_specific = turb_load.consumable_labour
G_air = turb_load.power / N_e_specific

with open(os.path.join(config.output_dirname, 'comp_turbine_ave_line.avl'), 'rb') as f:
    data = pickle.load(f)

turbine: Turbine = data['turbine']

In [3]:
def save_results(compressor, H_t_rel_dist, eta_ad_stag_dist, c1_a_rel_dist,  name='compressor'):
    with open(os.path.join(config.output_dirname, name), 'wb') as f:
        pickle.dump([compressor, H_t_rel_dist, eta_ad_stag_dist, c1_a_rel_dist], f)

def load_results(name='compressor'):
    with open(os.path.join(config.output_dirname, name), 'rb') as f:
        comp, H_t_rel_dist, eta_ad_stag_dist, c1_a_rel_dist = pickle.load(f)
    return comp, H_t_rel_dist, eta_ad_stag_dist, c1_a_rel_dist


In [4]:
stage_num = 10
H_t_rel_dist = QuadraticBezier(0.32, 0.33, stage_num, angle1=np.radians(10), angle2=np.radians(10))
eta_ad_stag_dist = QuadraticBezier(0.85, 0.86, stage_num, angle1=np.radians(10), angle2=np.radians(10))
R_av_dist = lambda z: 0.5
k_h_dist=lambda z: 0.98
c1_a_rel_dist = QuadraticBezier(0.45, 0.51, stage_num, angle1=np.radians(10), angle2=np.radians(-1))
h_rk_rel_dist = lambda z: interp1d([0, stage_num - 1], [2.5, 1.6])(z).__float__()
h_na_rel_dist = lambda z: interp1d([0, stage_num - 1], [3.2, 1.8])(z).__float__()
delta_a_rk_rel_dist = lambda z: interp1d([0, stage_num - 1], [0.3, 0.5])(z).__float__()
delta_a_na_rel_dist = lambda z: interp1d([0, stage_num - 1], [0.3, 0.5])(z).__float__()

compressor = Compressor(work_fluid=Air(), stage_num=stage_num, 
                        const_diam_par_arr=[0.5 for _ in range(stage_num)], 
                        p0_stag=cycle_compressor.p_stag_in, 
                        T0_stag=cycle_compressor.T_stag_in, 
                        G=G_air, 
                        n=turbine.n, 
                        H_t_rel_arr=H_t_rel_dist.get_array(),
                        eta_ad_stag_arr=eta_ad_stag_dist.get_array(), 
                        R_av_arr=QuadraticBezier.get_array_from_dist(R_av_dist, stage_num), 
                        k_h_arr=QuadraticBezier.get_array_from_dist(k_h_dist, stage_num), 
                        c1_a_rel_arr=c1_a_rel_dist.get_array(), 
                        h_rk_rel_arr=QuadraticBezier.get_array_from_dist(h_rk_rel_dist, stage_num), 
                        h_na_rel_arr=QuadraticBezier.get_array_from_dist(h_na_rel_dist, stage_num), 
                        delta_a_rk_rel_arr=QuadraticBezier.get_array_from_dist(delta_a_rk_rel_dist, stage_num), 
                        delta_a_na_rel_arr=QuadraticBezier.get_array_from_dist(delta_a_na_rel_dist, stage_num), 
                        d1_in_rel1=0.5, 
                        zeta_inlet=0.04, 
                        zeta_outlet=0.35, 
                        c11_init=250, precision=1e-4)

In [5]:
class DistWidget:
    def __init__(self, dist: QuadraticCurve, label=''):
        self.dist = dist
        self.curve_type = widgets.Dropdown(
                        options=['Bezier', 'Spline'],
                        value='Bezier'
        )
        self.label = widgets.Label(value=label)
        self.y1 = widgets.widgets.FloatSlider(value=dist.y1, description='y1', step=0.01, 
                                              readout_format='.3f', min=0, max=1)
        self.y2 = widgets.widgets.FloatSlider(value=dist.y2, description='y2', step=0.01, 
                                              readout_format='.3f', min=0, max=1)
        self.curve_setting_type = widgets.Dropdown(options=['Angle', 'Central pole'], value='Angle')
        
        if dist.angle1 and dist.angle2:
            self.angle1 = widgets.FloatSlider(value=np.degrees(dist.angle1), min=-90, max=90, step=0.25, 
                                              description='Угол 1, град')
            self.angle2 =  widgets.FloatSlider(value=np.degrees(dist.angle1), min=-90, max=90, step=0.25, 
                                               description='Угол 2, град')
            self.y0 = widgets.FloatSlider(value=dist.y0, description='y0', step=0.01, disabled=True,
                                          readout_format='.3f', min=0, max=2)
            self.x0 = widgets.FloatSlider(value=dist.x0, description='x0', step=0.05, 
                                          disabled=True, readout_format='.3f', min=0, max=1)
            self.curve_setting_type.value = 'Angle'
        else:
            self.angle1 = widgets.FloatSlider(value=10, min=-90, max=90, step=0.25, 
                                              description='Угол 1, град', disabled=True)
            self.angle2 =  widgets.FloatSlider(value=10, min=-90, max=90, step=0.25, 
                                               description='Угол 2, град', disabled=True)
            self.y0 = widgets.FloatSlider(value=dist.y0, description='y0', step=0.01,
                                          readout_format='.3f', min=0, max=2)
            self.x0 = widgets.FloatSlider(value=dist.x0, description='x0', step=0.05, 
                                          disabled=True, readout_format='.3f', min=0, max=1)
            self.curve_setting_type.value = 'Central pole'
        self.curve_setting_type.observe(self.curve_setting_type_change, names='value')
        self.plot_btn = widgets.Button(description='Plot')
        self.plot_btn.on_click(self.plot_dist)
        self.vbox = widgets.VBox([self.label, self.curve_type, self.curve_setting_type, self.y1, self.y2, 
                                  self.angle1, self.angle2, self.x0, self.y0, self.plot_btn])
        self.out = widgets.Output()
        self.hbox = widgets.HBox([self.vbox, self.out])
    
    def plot_dist(self, b):
        with self.out:
            self.out.clear_output()
            
        with self.out:
            plt.figure(figsize=(8, 6))
            plt.title(self.label.value, fontsize=15)
            y_arr = self.dist.get_array()
            x_arr = np.linspace(1, compressor.stage_num, compressor.stage_num)
            plt.plot(x_arr, y_arr, lw=2, color='orange')
            plt.ylim(ymin=round(min(y_arr), 2)-0.1, ymax=round(max(y_arr), 2)+0.1)
            plt.grid()
            plt.xlim(1, compressor.stage_num + 1)
            plt.xlabel(r'$z$', fontsize=14)
            x_ticks = np.linspace(1, compressor.stage_num, compressor.stage_num)
            plt.xticks(x_ticks, [int(x) for x in x_ticks], fontsize=13)
            plt.yticks(fontsize=13)
            plt.show()
    
    def set_dist(self):
        if self.curve_setting_type.value == 'Angle':
            if self.curve_type.value == 'Bezier':
                self.dist = QuadraticBezier(float(self.y1.value), float(self.y2.value), compressor.stage_num, 
                                                angle1=np.radians(float(self.angle1.value)), 
                                                angle2=np.radians(float(self.angle2.value)))
            elif self.curve_type.value == 'Spline':
                self.dist = QuadraticSpline(float(self.y1.value), float(self.y2.value), compressor.stage_num, 
                                                angle1=np.radians(float(self.angle1.value)), 
                                                angle2=np.radians(float(self.angle2.value)))
        elif self.curve_setting_type.value == 'Central pole':
            if self.curve_type.value == 'Bezier':
                self.dist = QuadraticBezier(float(self.y1.value), float(self.y2.value), compressor.stage_num, 
                                        y0=float(self.y0.value), 
                                        x0=float(self.x0.value))
            elif self.curve_type.value == 'Spline':
                self.dist = QuadraticSpline(float(self.y1.value), float(self.y2.value), compressor.stage_num, 
                                        y0=float(self.y0.value), 
                                        x0=float(self.x0.value))
    
    def curve_setting_type_change(self, change):
        if change['new'] == 'Angle':
            self.x0.disabled = True
            self.y0.disabled = True
            self.angle1.disabled = False
            self.angle2.disabled = False           
        elif change['new'] == 'Central pole':
            self.x0.disabled = False
            self.y0.disabled = False
            self.angle1.disabled = True
            self.angle2.disabled = True
    
    def show(self):
        display(self.vbox)


In [6]:
T0_wg = widgets.FloatText(description=r'$T_0,\ К$', value=round(compressor.T0_stag, 2), disabled=True)
p0_wg = widgets.FloatText(description=r'$p_0,\ МПа$', value=round(compressor.p0_stag / 1e6, 4), disabled=True)
G_wg = widgets.FloatText(description=r'$G,\ кг/с$', value=round(compressor.G, 2), disabled=True)
n_wg = widgets.FloatText(description=r'$n,\ об/мин$', value=round(compressor.n, 2), disabled=True)

stage_num_wg = widgets.IntSlider(value=stage_num, min=3, max=20, step=1, description='Stage number')
const_diam_par_1_wg = widgets.FloatSlider(value=compressor.const_diam_par_arr[0], min=0, max=1, step=0.01, 
                                          description='Const diam 1')
const_diam_par_2_wg = widgets.FloatSlider(value=compressor.const_diam_par_arr[0], min=0, max=1, step=0.01, 
                                          description='Const diam 2')
stage_num_change_diam_wg = widgets.IntSlider(value=int(stage_num / 2), min=1, 
                                              max=20, step=1, description='Change stage')
R_av_wg = widgets.FloatSlider(value=R_av_dist(0), min=0, max=1, step=0.005, description=r'$R_{ср}$')
k_h_wg = widgets.FloatSlider(value=k_h_dist(0), min=0, max=1, step=0.005, description=r'$k_h$')
h_rk_rel_1_wg = widgets.FloatSlider(value=h_rk_rel_dist(0), min=0.5, max=5, step=0.1, description=r'$\bar{h}_{рк1}$')
h_rk_rel_2_wg = widgets.FloatSlider(value=h_rk_rel_dist(compressor.stage_num - 1), min=0.5, max=5, step=0.1, 
                                    description=r'$\bar{h}_{рк2}$')
h_na_rel_1_wg = widgets.FloatSlider(value=h_na_rel_dist(0), min=0.5, max=5, step=0.1, description=r'$\bar{h}_{на1}$')
h_na_rel_2_wg = widgets.FloatSlider(value=h_na_rel_dist(compressor.stage_num - 1), min=0.5, max=5, step=0.1, 
                                    description=r'$\bar{h}_{на2}$')
delta_a_rk_rel_1_wg = widgets.FloatSlider(value=delta_a_rk_rel_dist(0), min=0.05, max=1.0, step=0.05, 
                                          description=r'$\bar{\delta}_{a\ рк1}$')
delta_a_rk_rel_2_wg = widgets.FloatSlider(value=delta_a_rk_rel_dist(compressor.stage_num - 1), min=0.05, max=1, step=0.05, 
                                          description=r'$\bar{\delta}_{a\ рк2}$')
delta_a_na_rel_1_wg = widgets.FloatSlider(value=delta_a_na_rel_dist(0), min=0.05, max=1, step=0.05, 
                                          description=r'$\bar{\delta}_{a\ на1}$')
delta_a_na_rel_2_wg = widgets.FloatSlider(value=delta_a_na_rel_dist(compressor.stage_num - 1), min=0.05, max=1, step=0.05, 
                                          description=r'$\bar{\delta}_{a\ на2}$')
d1_in_rel1_wg = widgets.FloatSlider(value=compressor.d1_in_rel1, min=0.2, max=0.7, step=0.01, 
                                    description=r'$\bar{d}_{вт1}$')
zeta_inlet_wg = widgets.FloatSlider(value=compressor.zeta_inlet, min=0, max=0.5, step=0.01, description=r'$\zeta_{вх}$')
zeta_outlet_wg = widgets.FloatSlider(value=compressor.zeta_outlet, min=0, max=0.7, step=0.01, description=r'$\zeta_{вых}$')
H_t_rel_dist_wg = DistWidget(H_t_rel_dist, r'$\bar{H}_т$')
c1_a_rel_dist_wg = DistWidget(c1_a_rel_dist, r'$\bar{c}_{1a}$')
eta_ad_stag_dist_wg = DistWidget(eta_ad_stag_dist, r'$\eta_{ад}^*$')
compute_btn = widgets.Button(description='Compute')
set_btn = widgets.Button(description='Set')

save_name_wg = widgets.Text(description='File for storing results:', value='compressor.comp')
save_btn = widgets.Button(description='Save')
load_btn = widgets.Button(description='Load')

compass_par_file_wg = widgets.Text(description='Compass parametrs file:', value='compressor_compass.xls')
write_compass_par_file_btn = widgets.Button(description='Write')

In [7]:
def set_dist_widget(dist: QuadraticCurve, dist_wg: DistWidget):
    if type(dist) == QuadraticBezier:
        dist_wg.curve_type.value = 'Bezier'
    elif type(dist) == QuadraticSpline:
        dist_wg.curve_type == 'Spline'
    dist_wg.curve_setting_type.value == 'Angle'
    dist_wg.y1.value = dist.y1
    dist_wg.y2.value = dist.y2
    dist_wg.x0.value = dist.x0
    dist_wg.y0.value = dist.y0
    dist_wg.angle1.value = np.degrees(dist.angle1)
    dist_wg.angle2.value = np.degrees(dist.angle2)
    

def set_widget_fields(compressor, H_t_rel_dist, eta_ad_stag_dist, c1_a_rel_dist):
    stage_num_wg.value = compressor.stage_num
    const_diam_par_1_wg.value = compressor.const_diam_par_arr[0]
    const_diam_par_2_wg.value = compressor.const_diam_par_arr[compressor.stage_num - 1]

    change_stage = 1
    for i in range(compressor.stage_num - 1):
        if compressor.const_diam_par_arr[i + 1] != compressor.const_diam_par_arr[i]:
            change_stage = i + 2
    stage_num_change_diam_wg.value = change_stage 
    R_av_wg.value = compressor.R_av_arr[0]
    k_h_wg.value = compressor.k_h_arr[0]
    h_rk_rel_1_wg.value = compressor.h_rk_rel_arr[0]
    h_rk_rel_2_wg.value = compressor.h_rk_rel_arr[compressor.stage_num - 1]
    h_na_rel_1_wg.value = compressor.h_na_rel_arr[0]
    h_na_rel_2_wg.value = compressor.h_na_rel_arr[compressor.stage_num - 1]
    delta_a_rk_rel_1_wg.value = compressor.delta_a_rk_rel_arr[0]
    delta_a_rk_rel_2_wg.value = compressor.delta_a_rk_rel_arr[compressor.stage_num - 1]
    delta_a_na_rel_1_wg.value = compressor.delta_a_na_rel_arr[0]
    delta_a_na_rel_2_wg.value = compressor.delta_a_na_rel_arr[compressor.stage_num - 1]
    d1_in_rel1_wg.value = compressor.d1_in_rel1
    zeta_inlet_wg.value = compressor.zeta_inlet
    zeta_outlet_wg.value = compressor.zeta_outlet
    set_dist_widget(H_t_rel_dist, H_t_rel_dist_wg)
    set_dist_widget(eta_ad_stag_dist, eta_ad_stag_dist_wg)
    set_dist_widget(c1_a_rel_dist, c1_a_rel_dist_wg)
    

In [8]:
def set_compressor_params():
    compressor.stage_num = stage_num_wg.value
    const_dp1 = const_diam_par_1_wg.value
    const_dp2 = const_diam_par_2_wg.value
    change_stage = stage_num_change_diam_wg.value
    compressor.const_diam_par_arr = (
        [const_dp1 for _ in range(change_stage - 1)] + 
        [const_dp2 for _ in range(change_stage - 1, compressor.stage_num)]
    )
    compressor.R_av_arr = QuadraticCurve.get_array_from_dist(lambda z: R_av_wg.value, compressor.stage_num)
    
    compressor.k_h_arr = QuadraticCurve.get_array_from_dist(lambda z: k_h_wg.value, compressor.stage_num)
    compressor.h_rk_rel_arr = QuadraticCurve.get_array_from_dist(
        lambda z: interp1d([0, compressor.stage_num - 1], 
                           [h_rk_rel_1_wg.value, h_rk_rel_2_wg.value])(z).__float__(), compressor.stage_num
    )
    compressor.h_na_rel_arr = QuadraticCurve.get_array_from_dist(
        lambda z: interp1d([0, compressor.stage_num - 1], 
                           [h_na_rel_1_wg.value, h_na_rel_2_wg.value])(z).__float__(), compressor.stage_num
    )
    compressor.delta_a_rk_rel_arr = QuadraticCurve.get_array_from_dist(
        lambda z: interp1d([0, compressor.stage_num - 1], 
                           [delta_a_rk_rel_1_wg.value, delta_a_rk_rel_2_wg.value])(z).__float__(), compressor.stage_num
    )
    compressor.delta_a_na_rel_arr = QuadraticCurve.get_array_from_dist(
        lambda z: interp1d([0, compressor.stage_num - 1], 
                           [delta_a_na_rel_1_wg.value, delta_a_na_rel_2_wg.value])(z).__float__(), compressor.stage_num
    )
    compressor.d1_in_rel1 = d1_in_rel1_wg.value
    compressor.zeta_inlet = zeta_inlet_wg.value
    compressor.zeta_outlet = zeta_outlet_wg.value
    H_t_rel_dist_wg.set_dist()
    eta_ad_stag_dist_wg.set_dist()
    c1_a_rel_dist_wg.set_dist()
    compressor.H_t_rel_arr = H_t_rel_dist_wg.dist.get_array()
    compressor.eta_ad_stag_arr = eta_ad_stag_dist_wg.dist.get_array()
    compressor.c1_a_rel_arr = c1_a_rel_dist_wg.dist.get_array()
    
def on_set_btn_click(b):
    set_compressor_params()
    
def on_compute_btn_click(b):
    set_compressor_params()
    compressor.compute()

def on_save_btn_click(b):
    save_results(compressor, H_t_rel_dist_wg.dist, eta_ad_stag_dist_wg.dist, c1_a_rel_dist_wg.dist, name=save_name_wg.value)

def on_load_btn_click(b):
    compressor, H_t_rel_dist, eta_ad_stag_dist, c1_a_rel_dist = load_results(name=save_name_wg.value)
    set_widget_fields(compressor, H_t_rel_dist, eta_ad_stag_dist, c1_a_rel_dist)

def on_compass_par_file_btn_click(b):
    try:
        compressor.write_compass_parameter_file(fname=os.path.join(config.output_dirname, compass_par_file_wg.value), 
                                                prefix='comp')
    except AttributeError:
        pass

compute_btn.on_click(on_compute_btn_click)
set_btn.on_click(on_set_btn_click)
save_btn.on_click(on_save_btn_click)
load_btn.on_click(on_load_btn_click)
write_compass_par_file_btn.on_click(on_compass_par_file_btn_click)

# Задание исходных параметров

In [9]:
vbox0 = widgets.VBox([save_name_wg, save_btn, load_btn])
vbox025 = widgets.VBox([compass_par_file_wg, write_compass_par_file_btn])
hbox05 = widgets.HBox([vbox0, vbox025])
vbox05 = widgets.VBox([T0_wg, p0_wg, n_wg, G_wg])
vbox1 = widgets.VBox(
    [stage_num_wg, const_diam_par_1_wg, const_diam_par_2_wg, stage_num_change_diam_wg, 
     R_av_wg, k_h_wg, d1_in_rel1_wg]
)
vbox2 = widgets.VBox([h_rk_rel_1_wg, h_rk_rel_2_wg, h_na_rel_1_wg, h_na_rel_2_wg, zeta_inlet_wg])
vbox3 = widgets.VBox([delta_a_rk_rel_1_wg, delta_a_rk_rel_2_wg, delta_a_na_rel_1_wg, delta_a_na_rel_2_wg, zeta_outlet_wg])
hbox1 = widgets.HBox([vbox1, vbox2, vbox3])
dist_tab = widgets.Tab()
dist_tab.children = [H_t_rel_dist_wg.hbox, c1_a_rel_dist_wg.hbox, eta_ad_stag_dist_wg.hbox]
dist_tab.set_title(0, 'К-т напора')
dist_tab.set_title(1, 'К-т расхода')
dist_tab.set_title(2, 'КПД')
vbox = widgets.VBox([hbox05, vbox05, hbox1, dist_tab, set_btn, compute_btn])
vbox

INFO - ITERATION 1
INFO - Computing stage 1
INFO - Computing stage 2
INFO - Computing stage 3
INFO - Computing stage 4
INFO - Computing stage 5
INFO - Computing stage 6
INFO - Computing stage 7
INFO - Computing stage 8
INFO - Computing stage 9
INFO - Computing stage 10
INFO - Computing stage 11
INFO - Computing stage 12
INFO - Computing stage 13
INFO - Computing stage 14
INFO - Computing stage 15
INFO - Residual = 0.18116

INFO - ITERATION 2
INFO - Computing stage 1
INFO - Computing stage 2
INFO - Computing stage 3
INFO - Computing stage 4
INFO - Computing stage 5
INFO - Computing stage 6
INFO - Computing stage 7
INFO - Computing stage 8
INFO - Computing stage 9
INFO - Computing stage 10
INFO - Computing stage 11
INFO - Computing stage 12
INFO - Computing stage 13
INFO - Computing stage 14
INFO - Computing stage 15
INFO - Residual = 0.02860

INFO - ITERATION 3
INFO - Computing stage 1
INFO - Computing stage 2
INFO - Computing stage 3
INFO - Computing stage 4
INFO - Computing stage 5
IN

In [10]:
class StageWidget:
    def __init__(self):
        self.D1_out = widgets.FloatText(description=r'$D_{1к},\ мм$', disabled=True)
        self.u1_out = widgets.FloatText(description=r'$u_{1к},\ м/с$', disabled=True)
        self.c1_a_rel = widgets.FloatText(description=r'$\bar{c}_{1a}$', disabled=True)
        self.c1_a = widgets.FloatText(description=r'$c_{1a},\ м/с$', disabled=True)
        self.delta_c1_a_st = widgets.FloatText(description=r'$\Delta c_{1a\ ст},\ м/с$', disabled=True)
        self.delta_c1_a_rk = widgets.FloatText(description=r'$\Delta c_{1a\ рк},\ м/с$', disabled=True)
        self.c2_a = widgets.FloatText(description=r'$c_{2a},\ м/с$', disabled=True)
        self.c2_a_rel = widgets.FloatText(description=r'$\bar{c}_{2a}$', disabled=True)
        self.eta_ad_stag = widgets.FloatText(description=r'$\eta_{ад}^*$', disabled=True)
        self.H_t_rel = widgets.FloatText(description=r'$\bar{H}_т$', disabled=True)
        self.d1_in_rel = widgets.FloatText(description=r'$\bar{d}_{1}$', disabled=True)
        self.H_t = widgets.FloatText(description=r'$H_t,\ КДж/кг$', disabled=True)
        self.L_z = widgets.FloatText(description=r'$L_z,\ КДж/кг$', disabled=True)
        self.H_ad = widgets.FloatText(description=r'$H_{ад},\ КДж/кг$', disabled=True)
        self.delta_T_stag = widgets.FloatText(description=r'$\Delta T^*,\ К$', disabled=True)
        self.T1_stag = widgets.FloatText(description=r'$T_1^*,\ К$', disabled=True)
        self.pi_stag = widgets.FloatText(description=r'$\pi^*$', disabled=True)
        self.p1_stag = widgets.FloatText(description=r'$p_1^*\cdot 10^{-4},\ Па$', disabled=True)
        self.a_cr1 = widgets.FloatText(description=r'$a_{кр1},\ м/с$', disabled=True)
        self.r1_av_rel = widgets.FloatText(description=r'$\bar{r}_{ср1}$', disabled=True)
        self.c1_u_rel = widgets.FloatText(description=r'$\bar{c}_{u1}$', disabled=True)
        self.tan_alpha1 =widgets.FloatText(description=r'$\tan{\alpha_1}$', disabled=True)
        self.alpha1 = widgets.FloatText(description=r'$\alpha_1,\ град$', disabled=True)
        self.sin_alpha1 = widgets.FloatText(description=r'$\sin{\alpha_1}$', disabled=True)
        self.alpha2 = widgets.FloatText(description=r'$\alpha_2,\ град$', disabled=True)
        self.sin_alpha2 = widgets.FloatText(description=r'$\sin{\alpha_2}$', disabled=True)
        self.epsilon_rk = widgets.FloatText(description=r'$\epsilon_{рк}$', disabled=True)
        self.epsilon_na = widgets.FloatText(description=r'$\epsilon_{на}$', disabled=True)
        self.w1 = widgets.FloatText(description=r'$w_1,\ м/с$', disabled=True)
        self.c2 = widgets.FloatText(description=r'$c_2,\ м/с$', disabled=True)
        self.tau1 = widgets.FloatText(description=r'$\tau(\lambda_1, k)$', disabled=True)
        self.T1 = widgets.FloatText(description=r'$T_1,\ К$', disabled=True)
        self.a1 = widgets.FloatText(description=r'$a_1,\ м/с$', disabled=True)
        self.M_w1 = widgets.FloatText(description=r'$M_{w1}$', disabled=True)
        self.lam2 = widgets.FloatText(description=r'$\lambda_{с2}$', disabled=True)
        self.h_rk = widgets.FloatText(description=r'$h_{рк},\ мм$', disabled=True)
        self.h_na = widgets.FloatText(description=r'$h_{на},\ мм$', disabled=True)
        self.b_a_rk = widgets.FloatText(description=r'$b_{a\ рк},\ мм$', disabled=True)
        self.b_a_na = widgets.FloatText(description=r'$b_{a\ на},\ мм$', disabled=True)
        self.h_rk_rel = widgets.FloatText(description=r'$\bar{h}_{рк}$', disabled=True)
        self.h_na_rel = widgets.FloatText(description=r'$\bar{h}_{на}$', disabled=True)
        self.w2 = widgets.FloatText(description=r'$w_2,\ м/с$', disabled=True)
        self.c1 = widgets.FloatText(description=r'$c_1,\ м/с$', disabled=True)
        
        self.vbox1 = widgets.VBox([self.D1_out, self.u1_out, self.c1_a_rel, self.c1_a, self.delta_c1_a_st, 
                                   self.delta_c1_a_rk, self.c2_a, self.c2_a_rel, self.eta_ad_stag, self.H_t_rel,
                                   self.d1_in_rel, self.H_t, self.L_z, self.H_ad])
        
        self.vbox2 = widgets.VBox([self.delta_T_stag, self.T1_stag, self.pi_stag, self.p1_stag, self.a_cr1, 
                                   self.r1_av_rel, self.c1_u_rel, self.tan_alpha1, self.alpha1, self.sin_alpha1, self.alpha2, 
                                   self.sin_alpha2, self.epsilon_rk, self.epsilon_na])
        
        self.vbox3 = widgets.VBox([ self.w1, self.w2, self.c1, self.c2, self.tau1, self.T1, self.a1, self.M_w1, self.lam2, 
                                   self.h_rk, self.h_na, self.b_a_rk, self.b_a_na, self.h_rk_rel, self.h_na_rel])
        self.hbox = widgets.HBox([self.vbox1, self.vbox2, self.vbox3])
    
    def update(self, stage: Stage):
        self.D1_out.value = round(stage.geom.D1_out * 1e3, 2)
        self.u1_out.value = round(stage.u1_out, 3)
        self.c1_a_rel.value = round(stage.c1_a_rel, 4)
        self.c1_a.value = round(stage.c1_a, 2)
        self.delta_c1_a_st.value = round(stage.c1_a - stage.c3_a, 2)
        self.delta_c1_a_rk.value = round(stage.c1_a - stage.c2_a, 2)
        self.c2_a.value = round(stage.c2_a, 2)
        self.c2_a_rel.value = round(stage.c2_a_rel, 4)
        self.eta_ad_stag.value = round(stage.eta_ad_stag, 3)
        self.H_t_rel.value = round(stage.H_t_rel, 3)
        self.d1_in_rel.value = round(stage.geom.d1_in_rel, 4)
        self.H_t.value = value=round(stage.H_t / 1e3, 2)
        self.L_z.value = round(stage.L_z / 1e3, 2)
        self.H_ad.value = round(stage.H_ad / 1e3, 2)
        self.delta_T_stag.value = round(stage.delta_T_stag, 2)
        self.T1_stag.value = round(stage.T1_stag, 2)
        self.pi_stag.value = round(stage.pi_stag, 3)
        self.p1_stag.value = round(stage.p1_stag / 1e4, 2)
        self.a_cr1.value = round(stage.a_cr1, 2)
        self.r1_av_rel.value = round(stage.geom.r1_av_rel, 3)
        self.c1_u_rel.value = round(stage.c1_u_rel, 3)
        self.tan_alpha1.value = round(np.tan(stage.alpha1), 3)
        self.alpha1.value = round(np.degrees(stage.alpha1), 2)
        self.sin_alpha1.value = round(np.sin(stage.alpha1), 3)
        self.alpha2.value = round(np.degrees(stage.alpha2), 2)
        self.sin_alpha2.value = round(np.sin(stage.alpha2), 3)
        self.epsilon_rk.value = round(np.degrees(stage.epsilon_rk), 2)
        self.epsilon_na.value = round(np.degrees(stage.epsilon_na), 2)
        self.w1.value = round(stage.w1, 2)
        self.c2.value = round(stage.c2, 2)
        self.tau1.value = round(stage.tau1, 3)
        self.T1.value = round(stage.T1, 2)
        self.a1.value = round(stage.a1, 2)
        self.M_w1.value = round(stage.M_w1_av, 3)
        self.lam2.value = round(stage.lam2, 3)
        self.h_rk.value = round(stage.geom.h_rk * 1e3, 1)
        self.h_na.value = round(stage.geom.h_na * 1e3, 1)
        self.b_a_rk.value = round(stage.geom.b_a_rk * 1e3, 1)
        self.b_a_na.value = round(stage.geom.b_a_na * 1e3, 1)
        self.h_rk_rel.value = round(stage.geom.h_rk_rel, 3)
        self.h_na_rel.value = round(stage.geom.h_na_rel, 3)
        self.w2.value = round(stage.w2, 2)
        self.c1.value = round(stage.c1, 2)
        

In [11]:
class CompressorWidget:
    def __init__(self, compressor):
        self.compressor = compressor
        self.stage_num = widgets.Dropdown(options=[str(i + 1) for i in range(compressor.stage_num)], value='1', 
                                                     description='Stage number')
        self.stage_widget = StageWidget()
        self.stage_num.observe(self._on_change_stage_num, names='value')
        self.vbox = widgets.VBox([self.stage_num, self.stage_widget.hbox])
    
    def _on_change_stage_num(self, change):
        try:
            stage_num = int(change['new']) - 1
            self.stage_widget.update(self.compressor[stage_num])
        except TypeError:
            pass
    
    def update(self):
        self.stage_num.options = [str(i + 1) for i in range(compressor.stage_num)]
        try:
            self.stage_widget.update(self.compressor[int(self.stage_num.value) - 1])
        except TypeError:
            pass
    

# Просмотр результатов

In [12]:
k_av_wg = widgets.FloatText(description=r'$k_{ср}$', value=0)
c_p_av_wg = widgets.FloatText(description=r'$c_{pср},\ Дж/(кг\cdot К)$', value=0)
pi_la_stag_wg = widgets.FloatText(description=r'$\pi_{ла}^*$', value=0)
eta_la_stag_wg = widgets.FloatText(description=r'$\eta_{ла}^*$', value=0)
sigma_inlet_wg = widgets.FloatText(description=r'$\sigma_{вх}$', value=0)
sigma_outlet_wg = widgets.FloatText(description=r'$\sigma_{вых}$', value=0)
pi_c_stag_wg = widgets.FloatText(description=r'$\pi_{к}^*$', value=0)
eta_c_stag_wg = widgets.FloatText(description=r'$\eta_{к}^*$', value=0)
eta_c_stag_p_wg = widgets.FloatText(description=r'$\eta_{кп}^*$', value=0)
vbox_int_par = widgets.VBox([k_av_wg, c_p_av_wg, pi_la_stag_wg, eta_la_stag_wg, sigma_inlet_wg, sigma_outlet_wg, 
                     pi_c_stag_wg, eta_c_stag_wg, eta_c_stag_p_wg])
compressor_wg = CompressorWidget(compressor)

fig_width_wg = widgets.FloatSlider(description='Figure width:', value=8, min=2, max=20, step=0.5)
fig_height_wg = widgets.FloatSlider(description='Figure height:', value=6, min=2, max=20, step=0.5)
value_to_plot_wg = widgets.Dropdown(options=['Stagnation temperature', 'Stagnation pressure', 
                                             'Axial velocity', 'Geometry'], 
                                    value='Stagnation temperature', description='Value to plot: ')
plot_value_dist_btn = widgets.Button(description='Plot')
out = widgets.Output()

plot_vbox = widgets.VBox([value_to_plot_wg, fig_width_wg, fig_height_wg, plot_value_dist_btn, out])

accordion = widgets.Accordion([vbox_int_par, compressor_wg.vbox, plot_vbox])
accordion.set_title(0, 'Интегральные параметры')
accordion.set_title(1, 'Параметры ступеней')
accordion.set_title(2, 'Графики')

update_res_btn = widgets.Button(description='Update')

def on_value_to_plot_value_change(change):
    if change['new'] == 'Stagnation temperature':
        fig_width_wg.value = 8
        fig_height_wg.value = 6
    elif change['new'] == 'Stagnation pressure':
        fig_width_wg.value = 8
        fig_height_wg.value = 6
    elif change['new'] == 'Axial velocity':
        fig_width_wg.value = 8
        fig_height_wg.value = 6
    elif change['new'] == 'Geometry':
        fig_width_wg.value = 15
        fig_height_wg.value = 6

def on_update_res_btn_click(b):
    compressor_wg.update()
    try:
        k_av_wg.value = round(compressor.k_av, 3)
        c_p_av_wg.value = round(compressor.c_p_av, 1)
        pi_la_stag_wg.value = round(compressor.pi_la_stag, 3)
        eta_la_stag_wg.value = round(compressor.eta_la_stag, 4)
        sigma_inlet_wg.value = round(compressor.sigma_inlet, 3)
        sigma_outlet_wg.value = round(compressor.sigma_outlet, 3)
        pi_c_stag_wg.value = round(compressor.pi_c_stag, 3)
        eta_c_stag_wg.value = round(compressor.eta_c_stag, 4)
        eta_c_stag_p_wg.value = round(compressor.eta_c_stag_p, 4)
    except AttributeError:
        pass

def on_plot_value_dist_btn_click(b):
    with out:
        out.clear_output()
    with out:
        if value_to_plot_wg.value == 'Stagnation temperature':
            compressor.plot_temp_dist(figsize=(fig_width_wg.value, fig_height_wg.value))
        elif value_to_plot_wg.value == 'Stagnation pressure':
            compressor.plot_press_dist(figsize=(fig_width_wg.value, fig_height_wg.value))
        elif value_to_plot_wg.value == 'Axial velocity':
            compressor.plot_c_a_dist(figsize=(fig_width_wg.value, fig_height_wg.value))
        elif value_to_plot_wg.value == 'Geometry':
            compressor.plot_geometry(figsize=(fig_width_wg.value, fig_height_wg.value))
            
plot_value_dist_btn.on_click(on_plot_value_dist_btn_click)
value_to_plot_wg.observe(on_value_to_plot_value_change, names='value')
update_res_btn.on_click(on_update_res_btn_click)

vbox = widgets.VBox([accordion, update_res_btn])
vbox

In [18]:
np.degrees(compressor[0].beta1)

43.013786162455055

In [17]:
compressor[0].geom.D1_av

0.50795554583509783